In [1]:
from selenium import webdriver
import pandas as pd
import os
import time
import requests
from selenium.webdriver.support.select import Select
import numpy as np
import json as js
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import date

In [11]:
# Advanced Stats Pull

season = "2020-21"

path_to_chromedriver = r"C:\Users\asus\Documents\Development\chromedriver.exe"  # Path to access a chrome driver
browser = webdriver.Chrome(executable_path=path_to_chromedriver)

url = (
    "https://www.nba.com/stats/players/traditional/?Season="
    + season
    + "&SeasonType=Regular%20Season"
)
browser.get(url)



In [3]:
# Select the page
pages = Select(browser.find_element_by_xpath('//select[@ng-model="vm.data.page"]'))
pages.select_by_visible_text('All')


In [4]:
# Get Table Headers

count = 1
headers = []
while True:
    try:
        tableHeader = browser.find_element_by_xpath(
            '//table[@class = "table"]/thead/tr/th[' + str(count) + "]"
        )
        # print(tableHeader.text)
        headers.append(tableHeader.text)
        count += 1
    except:
        print("No more headers")
        break

headers

No more headers


['PLAYER',
 'TEAM',
 'GP',
 'W',
 'L',
 'MIN',
 'PTS',
 'DRIVE\nPTS',
 'DRIVE\nFG%',
 'C&S\nPTS',
 'C&S\nFG%',
 'PULL UP\nPTS',
 'PULL UP\nFG%',
 'PAINT\nTOUCH PTS',
 'PAINT\nTOUCH FG%',
 'POST\nTOUCH PTS',
 'POST\nTOUCH FG%',
 'ELBOW\nTOUCH PTS',
 'ELBOW\nTOUCH FG%',
 'EFG%']

In [5]:
count = 1
playerData = []
while True:
    try:
        tableBody = browser.find_element_by_xpath('//table[@class = "table"]/tbody/tr[' + str(count) + ']')
        data = tableBody.text.split('\n')[1].split()
        playerData.append(tableBody.text)
        count += 1
    except:
        print("No more headers")
        break


No more headers


In [6]:
# Read the table body contents
tableBody = browser.find_element_by_xpath('//table[@class = "table"]/tbody')

In [7]:
# Create dataframe from each row of data

tableData = tableBody.text.split("\n")
count = 0
players = pd.DataFrame()
for i in range(0, len(tableData[0::2])):
    playerData = pd.DataFrame(tableData[count + 1].split())
    playerName = pd.DataFrame([tableData[count]])
    player = pd.concat([playerName, playerData])
    player = player.transpose()
    players = players.append(player)
    count += 2


players.columns = headers
players

,PLAYER,TEAM,GP,W,L,MIN,PTS,DRIVE\nPTS,DRIVE\nFG%,C&S\nPTS,C&S\nFG%,PULL UP\nPTS,PULL UP\nFG%,PAINT\nTOUCH PTS,PAINT\nTOUCH FG%,POST\nTOUCH PTS,POST\nTOUCH FG%,ELBOW\nTOUCH PTS,ELBOW\nTOUCH FG%,EFG%
0,Aaron Gordon,ORL,12,6,6,27.6,14.7,2.8,46.2,3.3,42.4,3.3,32.0,3.1,70.8,1.5,31.3,1.0,71.4,53.3
0,Aaron Holiday,IND,13,8,5,21.1,5.9,2.2,46.7,1.8,25.0,0.5,14.3,0.0,0.0,0.0,0.0,0.2,50.0,41.4
0,Aaron Nesmith,BOS,5,3,2,13.2,2.6,0.4,33.3,0.6,8.3,1.6,42.9,0.4,100,0.0,0.0,0.4,100,28.3
0,Abdel Nader,PHX,1,0,1,3.3,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,Adam Mokoka,CHI,5,0,5,7.8,2.2,0.4,33.3,0.6,14.3,0.0,0.0,0.4,100,0.0,0.0,0.0,0.0,39.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Yogi Ferrell,CLE,2,0,2,20.3,9.5,3.0,37.5,3.0,50.0,4.5,30.8,0.0,0.0,0.0,0.0,1.0,100,45.2
0,Yuta Watanabe,TOR,7,4,3,10.9,2.0,0.0,0.0,1.7,44.4,0.0,0.0,0.3,25.0,0.0,0.0,0.0,0.0,38.9
0,Zach LaVine,CHI,13,5,8,35.5,26.9,8.8,57.1,4.2,40.4,9.2,39.7,0.8,50.0,0.2,33.3,0.5,60.0,57.6
0,Zeke Nnaji,DEN,5,4,1,2.2,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.4,50.0,0.0,0.0,0.0,0.0,25.0


In [8]:
# Get the type of stats being parsed and create file title
statsType = browser.find_element_by_xpath('//ol[@class="breadcrumbs__list"]')
statsType = statsType.text.split("/")
statsFile = ""
for stat in statsType[1:]:
    statsFile = statsFile + stat.replace(" ", "") + "_"

today = date.today()
statsFile = season + "_" + statsFile + str(today)

In [9]:
# Convert df to csv
pathName = "C:\\Users\\asus\\OneDrive\\Documents\\Fantasy BBall\\Fantasy-Basketball-ESPN\\NBAcomStats\\"
fileName = pathName + statsFile + ".csv"
player.to_csv(fileName)

In [10]:
browser.quit()